<a href="https://colab.research.google.com/github/HpDiniz/Leitor-de-PDF/blob/main/PDF_Extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install pdfminer
!pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
import io
import re
import os
import PyPDF2
from io import StringIO
from google.colab import files
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter

In [11]:
file_objects = files.upload()

Saving Extrato XP - Cliente 01.pdf to Extrato XP - Cliente 01 (1).pdf


In [12]:
# Converte o conteúdo do PDF para texto
def convert_pdf_to_txt(path):

    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()

    return text

def remove_header_and_footer(text, cm, tm, fontDict, fontSize):
    y = tm[5]
    if y > 50 and y < 720:
        parts.append(text)

In [13]:
text_body = ""
text_miner = ""

for file in list(file_objects.keys()):

    file_name, file_extension = os.path.splitext(file)

    if file_extension != '.pdf':
        continue

    # Lê o conteúdo do arquivo PDF
    text_miner = convert_pdf_to_txt(file)
    
    reader = PyPDF2.PdfReader(file)

    for page in reader.pages:

        parts = []

        page.extract_text(visitor_text=remove_header_and_footer)
        text_body = text_body + "".join(parts)

In [58]:
text_miner

'Extrato de Cotista\n\nConsolidado\n\nMovimentação de 15/05/2022 a 11/11/2022\n\nNome do Cliente\nEndereço\n\nCEP: 02440-00\n\nPOSIÇÃO CONSOLIDADA\n\nFundo\n\nCota\n\nQuantidade\n\nValor Aplicado\n\nValor Bruto\n\nIR\n\nSulAmérica Inflatie FI Renda Fixa Longo Prazo\n\nXP Referenciado FI Referenciado DI CP\n\nXPA MM II FIC FIM\n\nXPA RV FIC FIM\n\nXPA Vértice Crédito FIM CP\n\n49,03907260\n\n2,55168782\n\n1,48495549\n\n1,51471586\n\n1,31704969\n\n485,46020696\n\n9.607,86926679\n\n206.015,91600648\n\n86.244,99799744\n\n62.558,82325043\n\n21.434,59\n\n23.303,62\n\n231.165,15\n\n119.600,00\n\n67.664,32\n\n23.806,52\n\n24.516,28\n\n305.924,47\n\n130.636,67\n\n82.393,08\n\n13,80\n\n272,84\n\n2.095,63\n\n1.655,50\n\n816,74\n\nEmissão: 11/11/2022\n\nCódigo: 3Nxxx7\n\nIOF\n\n0,00\n\n0,00\n\n0,00\n\n0,00\n\n0,00\n\nValor Liquido\n\n23.792,72\n\n24.243,44\n\n303.828,84\n\n128.981,17\n\n81.576,34\n\nTotal na Instituição\n\n463.167,68\n\n567.277,02\n\n4.854,51\n\n0,00\n\n562.422,51\n\nSulAmérica In

In [124]:
import re

# A primeira regex obtém o texto situado entre o termo "POSIÇÃO CONSOLIDADA" e o termo "Emissão"
regex_1 = "(?<=POSI..O CONSOLIDADA).*?(?=Emissão:)"
result_1 = str(re.search(regex_1, text_miner, flags=(re.IGNORECASE | re.DOTALL)).group(0))

# A segunda regex obtém todas as linhas que possuírem pelo menos uma letra e forem diferente dos valores que
# aparecem nos headers das tabelas: "Fundo", "Cota", "Quantidade", "Valor Aplicado", etc...
regex_2 = "^(?!Fundo|Cota|Quantidade|Valor Aplicado|Valor Bruto|IR|IOF|Valor|Liquido$)^.*[a-zA-Z].*$"
result_2 = re.findall(regex_2, result_1, re.MULTILINE)

print("ATIVOS ENCONTRADOS:")
dictionary = {}
for ativo in result_2:
    print(" - " + ativo)
    dictionary[ativo.replace(" ","")] = ativo

ATIVOS ENCONTRADOS:
 - SulAmérica Inflatie FI Renda Fixa Longo Prazo
 - XP Referenciado FI Referenciado DI CP
 - XPA MM II FIC FIM
 - XPA RV FIC FIM
 - XPA Vértice Crédito FIM CP


In [125]:
dictionary

{'SulAméricaInflatieFIRendaFixaLongoPrazo': 'SulAmérica Inflatie FI Renda Fixa Longo Prazo',
 'XPReferenciadoFIReferenciadoDICP': 'XP Referenciado FI Referenciado DI CP',
 'XPAMMIIFICFIM': 'XPA MM II FIC FIM',
 'XPARVFICFIM': 'XPA RV FIC FIM',
 'XPAVérticeCréditoFIMCP': 'XPA Vértice Crédito FIM CP'}

In [121]:
print(text_body)


POSIÇÃOCONSOLIDADA
Fundo Cota Quantidade ValorAplicado ValorBruto IR IOF ValorLiquido
SulAméricaInflatieFIRendaFixaLongoPrazo 49,03907260 485,46020696 21.434,59 23.806,52 13,80 0,00 23.792,72
XPReferenciadoFIReferenciadoDICP 2,55168782 9.607,86926679 23.303,62 24.516,28 272,84 0,00 24.243,44
XPAMMIIFICFIM 1,48495549 206.015,91600648 231.165,15 305.924,47 2.095,63 0,00 303.828,84
XPARVFICFIM 1,51471586 86.244,99799744 119.600,00 130.636,67 1.655,50 0,00 128.981,17
XPAVérticeCréditoFIMCP 1,31704969 62.558,82325043 67.664,32 82.393,08 816,74 0,00 81.576,34
TotalnaInstituição 463.167,68 567.277,02 4.854,51 0,00 562.422,51
SulAméricaInflatieFIRendaFixaLongoPrazo
Histórico DataMov. Cotização Cota Quantidade ValorBruto IR IOF ValorLiquido
SaldoAnterior 15/05/2022 48,44468460 2.001,43698264 96.958,98 0,00 0,00 96.958,98
Resgateparcial 25/05/2022 26/05/2022 48,74387760 474,93144042 23.150,00 0,00 0,00 23.150,00
Resgateparcial 04/08/2022 05/08/2022 48,98922100 1.041,04533526 51.000,00 21,82 0,0

In [112]:
import pandas as pd
  
# initialize data of lists.
data = {
    "CLASSIFICAÇÃO": [],
    "ATIVO": [],
    "EXPOSIÇÃO": [],
    "TIT/INST": [],
    "ATUAL": [],
    "DATA": [],
    "ON/OFF": [],
    "DATA DE VENCIMENTO": []
}
  
# Create DataFrame
df = pd.DataFrame(data)
  
# Print the output.
df

,CLASSIFICAÇÃO,ATIVO,EXPOSIÇÃO,TIT/INST,ATUAL,DATA,ON/OFF,DATA DE VENCIMENTO
